<a href="https://colab.research.google.com/github/smit-patel-m/SummerInternship/blob/main/PUF_GBNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
import pandas as pd
from google.colab import drive
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F


from sklearn.base import BaseEstimator

In [116]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [117]:
import os
from google.colab import drive

if not os.path.exists("/content/db1"):
    drive.mount('/content/drive')
    !unzip "/content/drive/MyDrive/Summer Internship/ML Attack.zip" -d "/content/db1"
else:
    print("Folder '/content/db1' already exists. Skipping unzip.")


Folder '/content/db1' already exists. Skipping unzip.


# Load the Excel files into pandas DataFrames

In [118]:
# Load the Excel files into pandas DataFrames

response_delay_path = "/content/db1/ML Attack/Responce_Delay_2us.xlsx"
challenge_path = "/content/db1/ML Attack/Challenge.xlsx"
response_delay_sheet_names = pd.ExcelFile(response_delay_path).sheet_names
response_delay_df = pd.read_excel(response_delay_path, sheet_name=0,header=None)

challenge_df = pd.read_excel(challenge_path,header=None)


In [119]:
# Display the first few rows of each DataFrame to verify
print("Selected Sheet Name:", response_delay_sheet_names)

print("Response Delay DataFrame:")
display(response_delay_df.head())

print("\nChallenge DataFrame:")
display(challenge_df.head())

Selected Sheet Name: ['FD_MODE_1', 'VD_MODE_1', 'FD_MODE_2', 'VD_MODE_2', 'FD_MODE_3', 'VD_MODE_3', 'FD_MODE_4', 'VD_MODE_4', 'FD_MODE_5', 'VD_MODE_5', 'FD_MODE_6', 'VD_MODE_6', 'FD_MODE_7', 'VD_MODE_7', 'FD_MODE_8', 'VD_MODE_8']
Response Delay DataFrame:


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0,1,1,1,1,0,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
1,1,0,1,1,1,0,0,0,0,1,...,0,1,1,0,0,0,1,0,0,1
2,1,1,0,0,1,1,0,0,0,0,...,1,1,0,1,1,0,1,1,1,1
3,1,1,1,0,1,1,0,0,1,1,...,1,1,0,0,0,1,1,0,1,1
4,0,1,0,0,0,1,0,1,1,1,...,1,0,1,1,0,0,0,1,0,0



Challenge DataFrame:


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0,1,0,0,0,0,0,1,1,1,...,1,0,1,1,0,0,0,1,1,1
1,0,1,1,0,0,1,0,1,1,1,...,0,1,1,0,1,0,0,1,0,0
2,0,0,1,0,1,0,0,1,0,0,...,0,0,1,1,0,1,1,0,1,1
3,0,0,0,1,1,0,0,0,1,0,...,1,1,0,0,1,0,0,0,1,1
4,1,1,1,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,1,0


In [120]:
# Convert to NumPy
X = challenge_df.values.astype(np.float32)     # shape: (N, 32)
Y = response_delay_df.values.astype(np.float32)  # shape: (N, 32)

In [121]:
X

array([[0., 1., 0., ..., 1., 1., 1.],
       [0., 1., 1., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 1., 1.],
       ...,
       [0., 1., 1., ..., 1., 1., 1.],
       [1., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [122]:
Y

array([[0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 1.],
       [1., 1., 0., ..., 1., 1., 1.],
       ...,
       [0., 1., 0., ..., 1., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 1., 0., ..., 1., 1., 1.]], dtype=float32)

In [123]:
#X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [124]:

import re


def get_next_model_version(base_path="/content/drive/MyDrive/SummerIntern/Puf Models", prefix="puf_gbnn"):
    os.makedirs(base_path, exist_ok=True)
    existing = [f for f in os.listdir(base_path) if f.startswith(prefix)]
    versions = [int(re.findall(r'V(\d+)', name)[0]) for name in existing if re.search(r'V\d+', name)]
    next_version = max(versions) + 1 if versions else 1
    model_folder = f"{prefix}_V{next_version}"
    return os.path.join(base_path, model_folder)

def save_gbnn_model(model, path, history=None):
    os.makedirs(path, exist_ok=True)
    model_path = os.path.join(path, "model.pt")
    torch.save(model.state_dict(), model_path)
    print(f"✅ GBNN model saved at: {model_path}")

    if history is not None:
        history_df = pd.DataFrame(history, columns=[
            "Stage", "Epoch", "Train Accuracy", "Val Accuracy"
        ])
        excel_path = os.path.join(path, "training_history.xlsx")
        history_df.to_excel(excel_path, index=False)
        print(f" Training history saved at: {excel_path}")

def load_gbnn_model(model_class, model_args, path):
    model_path = os.path.join(path, "model.pt")
    model = model_class(**model_args)
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
    print(f"✅ GBNN model loaded from: {model_path}")
    return model


In [125]:


# MLP for each stage
class MultiOutputNN(nn.Module):
    def __init__(self):
        super(MultiOutputNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )


    def forward(self, x):
        return self.model(x)
def bitwise_accuracy(preds, labels):
    return (preds == labels).float().mean().item()

def exact_match(preds, labels):
    return ((preds == labels).all(dim=1)).float().mean().item()




class GBNN(nn.Module):
    def __init__(self, n_stages=30):
        super(GBNN, self).__init__()
        self.n_stages = n_stages
        self.models = nn.ModuleList([MultiOutputNN() for _ in range(n_stages)])

    def forward(self, x):
        out = torch.zeros(x.size(0), 32).to(x.device)
        for model in self.models:
            out += model(x)
        return out

    def stagewise_forward(self, x, up_to_stage):
        out = torch.zeros(x.size(0), 32).to(x.device)
        for model in self.models[:up_to_stage]:
            out += model(x)
        return out


In [126]:

def split_dataset(dataset):
    total_size = len(dataset)
    train_size = int(0.7 * total_size)
    test_size = int(0.2 * total_size)
    val_size = total_size - train_size - test_size
    return random_split(dataset, [train_size, test_size, val_size])


def train_gbnn(model, X_train, y_train, X_val, y_val, X_test, y_test, n_epochs=200, lr=0.01, save_path=None):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_train, y_train = X_train.to(device), y_train.to(device)
    X_val, y_val = X_val.to(device), y_val.to(device)
    X_test, y_test = X_test.to(device), y_test.to(device)

    criterion = nn.MSELoss()  # Or use nn.BCEWithLogitsLoss()

    history = []

    for stage_idx, sub_model in enumerate(model.models):
        print(f"\n🎯 Stage {stage_idx + 1}/{model.n_stages}")
        optimizer = optim.Adam(sub_model.parameters(), lr=lr)

        for epoch in range(n_epochs):
            model.train()
            with torch.no_grad():
                logits_so_far = model.stagewise_forward(X_train, stage_idx)
                pred_prob = torch.sigmoid(logits_so_far)
                residual = y_train - pred_prob

            pred_logits = sub_model(X_train)
            loss = criterion(pred_logits, residual)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Evaluate train/val accuracy & log loss EVERY epoch
            model.eval()
            with torch.no_grad():
                train_logits = model.forward(X_train)
                train_preds = (torch.sigmoid(train_logits) > 0.5).float()
                train_acc = bitwise_accuracy(train_preds, y_train)

                val_logits = model.forward(X_val)
                val_preds = (torch.sigmoid(val_logits) > 0.5).float()
                val_acc = bitwise_accuracy(val_preds, y_val)


            history.append([
                stage_idx + 1,
                epoch + 1,
                float(train_acc),
                float(val_acc),
                float(loss.item())
            ])


            if (epoch % 100 == 0) or (epoch == n_epochs - 1):
                print(f"Epoch {epoch+1}/{n_epochs} | Train Loss: {loss.item():.4f} | "
                      f"Train Acc: {train_acc*100:.2f}% | Val Acc: {val_acc*100:.2f}%")


    if save_path is not None:
        os.makedirs(save_path, exist_ok=True)
        # Save model
        torch.save(model.state_dict(), os.path.join(save_path, "model.pt"))
        print(f"✅ GBNN model saved at: {os.path.join(save_path, 'model.pt')}")
        # Save Excel history
        import pandas as pd
        history_df = pd.DataFrame(history, columns=[
            "Stage", "Epoch", "Train Accuracy", "Val Accuracy", "Train Error"
        ])
        history_df.to_excel(os.path.join(save_path, "training_history.xlsx"), index=False)
        print(f" Training history saved at: {os.path.join(save_path, 'training_history.xlsx')}")

    # Print final test performance
    model.eval()
    with torch.no_grad():
        test_logits = model.forward(X_test)
        test_preds = (torch.sigmoid(test_logits) > 0.5).float()
        test_acc = bitwise_accuracy(test_preds, y_test)
        test_exact = exact_match(test_preds, y_test)
    print(f"\n✅ Test Acc: {test_acc*100:.2f}% | Test Exact Match: {test_exact*100:.2f}%")

    return model, history



In [133]:


save_path = get_next_model_version(
    base_path="/content/drive/MyDrive/SummerIntern/Puf Models",
    prefix="puf_gbnn"
)
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)

dataset = TensorDataset(X_tensor, Y_tensor)
train_set, test_set, val_set = split_dataset(dataset)

train_data = next(iter(DataLoader(train_set, batch_size=len(train_set))))
test_data = next(iter(DataLoader(test_set, batch_size=len(test_set))))
val_data = next(iter(DataLoader(val_set, batch_size=len(val_set))))

# Unpack data
X_train, y_train = train_data
X_val, y_val = val_data
X_test, y_test = test_data
# Train and save
model = GBNN(n_stages=35)
trained_model, history = train_gbnn(model, X_train, y_train, X_val, y_val, X_test, y_test, n_epochs=500, lr=0.01, save_path=save_path)


🎯 Stage 1/35
Epoch 1/500 | Train Loss: 0.2584 | Train Acc: 50.64% | Val Acc: 50.61%
Epoch 101/500 | Train Loss: 0.2302 | Train Acc: 52.87% | Val Acc: 50.68%
Epoch 201/500 | Train Loss: 0.2235 | Train Acc: 53.60% | Val Acc: 50.64%
Epoch 301/500 | Train Loss: 0.2209 | Train Acc: 53.94% | Val Acc: 50.74%
Epoch 401/500 | Train Loss: 0.2191 | Train Acc: 54.16% | Val Acc: 50.80%
Epoch 500/500 | Train Loss: 0.2176 | Train Acc: 54.42% | Val Acc: 50.98%

🎯 Stage 2/35
Epoch 1/500 | Train Loss: 0.2506 | Train Acc: 54.30% | Val Acc: 51.14%
Epoch 101/500 | Train Loss: 0.2234 | Train Acc: 55.43% | Val Acc: 51.24%
Epoch 201/500 | Train Loss: 0.2178 | Train Acc: 56.21% | Val Acc: 51.24%
Epoch 301/500 | Train Loss: 0.2156 | Train Acc: 56.43% | Val Acc: 51.32%
Epoch 401/500 | Train Loss: 0.2138 | Train Acc: 56.67% | Val Acc: 51.00%
Epoch 500/500 | Train Loss: 0.2131 | Train Acc: 56.76% | Val Acc: 50.96%

🎯 Stage 3/35
Epoch 1/500 | Train Loss: 0.2330 | Train Acc: 56.85% | Val Acc: 51.14%
Epoch 101/500 |

In [128]:
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)

dataset = TensorDataset(X_tensor, Y_tensor)
train_set, test_set, val_set = split_dataset(dataset)

train_data = next(iter(DataLoader(train_set, batch_size=len(train_set))))
test_data = next(iter(DataLoader(test_set, batch_size=len(test_set))))
val_data = next(iter(DataLoader(val_set, batch_size=len(val_set))))

# Unpack data
X_train, y_train = train_data
X_val, y_val = val_data
X_test, y_test = test_data

# --- Train ---
model = GBNN(n_stages=35)
train_gbnn(model, X_train, y_train, X_val, y_val, X_test, y_test, n_epochs=200, lr=0.01)



🎯 Stage 1/35
Epoch 1/200 | Train Loss: 0.2573 | Train Acc: 50.85% | Val Acc: 50.86%
Epoch 101/200 | Train Loss: 0.2290 | Train Acc: 52.98% | Val Acc: 50.69%
Epoch 200/200 | Train Loss: 0.2230 | Train Acc: 53.68% | Val Acc: 51.01%

🎯 Stage 2/35
Epoch 1/200 | Train Loss: 0.2474 | Train Acc: 53.63% | Val Acc: 50.86%
Epoch 101/200 | Train Loss: 0.2249 | Train Acc: 55.03% | Val Acc: 50.91%
Epoch 200/200 | Train Loss: 0.2182 | Train Acc: 55.84% | Val Acc: 50.91%

🎯 Stage 3/35
Epoch 1/200 | Train Loss: 0.2391 | Train Acc: 55.72% | Val Acc: 50.93%
Epoch 101/200 | Train Loss: 0.2208 | Train Acc: 56.64% | Val Acc: 51.11%
Epoch 200/200 | Train Loss: 0.2163 | Train Acc: 57.23% | Val Acc: 51.38%

🎯 Stage 4/35
Epoch 1/200 | Train Loss: 0.2320 | Train Acc: 57.51% | Val Acc: 51.49%
Epoch 101/200 | Train Loss: 0.2102 | Train Acc: 58.99% | Val Acc: 51.54%
Epoch 200/200 | Train Loss: 0.2037 | Train Acc: 59.72% | Val Acc: 51.67%

🎯 Stage 5/35
Epoch 1/200 | Train Loss: 0.2236 | Train Acc: 59.82% | Val Acc

(GBNN(
   (models): ModuleList(
     (0-34): 35 x MultiOutputNN(
       (model): Sequential(
         (0): Linear(in_features=32, out_features=64, bias=True)
         (1): ReLU()
         (2): Linear(in_features=64, out_features=64, bias=True)
         (3): ReLU()
         (4): Linear(in_features=64, out_features=32, bias=True)
       )
     )
   )
 ),
 [[1, 1, 0.5085446238517761, 0.5086249709129333, 0.25726503133773804],
  [1, 2, 0.5094553828239441, 0.5086874961853027, 0.25095322728157043],
  [1, 3, 0.5090267658233643, 0.5068749785423279, 0.24902571737766266],
  [1, 4, 0.5086339116096497, 0.5062500238418579, 0.24778789281845093],
  [1, 5, 0.5086517930030823, 0.5061874985694885, 0.24748294055461884],
  [1, 6, 0.5086785554885864, 0.5056874752044678, 0.24736183881759644],
  [1, 7, 0.5089107155799866, 0.5058125257492065, 0.24722430109977722],
  [1, 8, 0.5093035697937012, 0.5057500004768372, 0.24707019329071045],
  [1, 9, 0.5096339583396912, 0.5068125128746033, 0.24690544605255127],
  [1, 

In [129]:

# Predict 32-bit output from 32-bit input
def predict_single_32bit(model: nn.Module, input_bits: list, threshold: float = 0.5) -> list:
    assert len(input_bits) == 32, "Input must be a 32-bit binary list."

    model.eval()
    input_array = np.array(input_bits, dtype=np.float32)
    normalized_input = input_array * 0.98 + 0.01

    with torch.no_grad():
        input_tensor = torch.tensor(normalized_input).unsqueeze(0)  # Shape: [1, 32]
        logits = model(input_tensor)
        probs = torch.sigmoid(logits)
        prediction = (probs > threshold).int().squeeze().tolist()

    return prediction

def predict_batch_32bit(model: nn.Module, input_array: np.ndarray, threshold: float = 0.5) -> list:
    assert input_array.ndim == 2 and input_array.shape[1] == 32, "Input must be of shape [N, 32]"

    model.eval()
    normalized_input = input_array * 0.98 + 0.01

    with torch.no_grad():
        input_tensor = torch.tensor(normalized_input, dtype=torch.float32)
        logits = model(input_tensor)
        probs = torch.sigmoid(logits)
        predictions = (probs > threshold).int().tolist()  # List of [0,1] lists

    return predictions


def evaluate_predictions(y_true: np.ndarray, y_pred: np.ndarray):

    assert y_true.shape == y_pred.shape, "Shape mismatch between true and predicted labels"

    metrics = {}

    # Bitwise metrics (macro = average over each bit)
    metrics["bitwise_accuracy"] = accuracy_score(y_true.flatten(), y_pred.flatten())
    metrics["macro_precision"] = precision_score(y_true, y_pred, average='macro', zero_division=0)
    metrics["macro_recall"] = recall_score(y_true, y_pred, average='macro', zero_division=0)
    metrics["macro_f1"] = f1_score(y_true, y_pred, average='macro', zero_division=0)

    # Hamming loss (fraction of incorrect bits)
    metrics["hamming_loss"] = hamming_loss(y_true, y_pred)

    # Exact match (all 32 bits correct)
    exact_matches = np.all(y_true == y_pred, axis=1)
    metrics["exact_match_ratio"] = np.mean(exact_matches)

    return metrics


In [130]:

# Predict a 32-bit binary input
input_bits = [0, 1, 0, 1, 0, 1, 1, 0,
              1, 0, 0, 1, 1, 0, 0, 1,
              1, 1, 0, 0, 1, 0, 1, 0,
              1, 1, 0, 0, 1, 0, 0, 1]

predicted_output = predict_single_32bit(model, input_bits)
print("\nPredicted Output Bits:", predicted_output)


Predicted Output Bits: [1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0]


In [131]:
X = challenge_df.values.astype(np.float32)  # shape [N, 32]

# Predict all rows
predictions = predict_batch_32bit(model, X)


predictions_np = np.array(predictions)

print("Predictions shape:", predictions_np.shape)
print("First prediction:", predictions[0])

Predictions shape: (5000, 32)
First prediction: [1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0]


In [132]:

Y_true = Y.astype(np.int32)
X_input = X.astype(np.float32)
Y_pred = np.array(predict_batch_32bit(model, X_input))

# Evaluate
metrics = evaluate_predictions(Y_true, Y_pred)


for name, value in metrics.items():
    print(f"{name}: {value:.4f}")


NameError: name 'accuracy_score' is not defined

In [ ]:

def top_k_bit_errors(y_true, y_pred, k=5):


    bit_errors_per_sample = np.sum(y_true != y_pred, axis=1)


    top_k_indices = np.argsort(bit_errors_per_sample)[-k:][::-1]

    top_k_errors = bit_errors_per_sample[top_k_indices]

    return top_k_indices, top_k_errors


top_indices, top_errors = top_k_bit_errors(Y_true, Y_pred, k=5)

print("Top 5 samples with highest bit errors:")
for rank, (idx, errors) in enumerate(zip(top_indices, top_errors), 1):
    print(f"Rank {rank}: Sample index {idx}, Bit errors = {errors}")
    print(f"True bits: {Y_true[idx]}")
    print(f"Pred bits: {Y_pred[idx]}")
    print("-" * 30)